In [1]:
import pandas as pd
import numpy as np
from sklearn.lda import LDA
from geopy.distance import vincenty
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

C:\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn import svm
from sklearn.grid_search import GridSearchCV

In [3]:
df = pd.read_csv("../assets/train.csv")
test = pd.read_csv("../assets/test.csv")
sample = pd.read_csv('../Assets/sampleSubmission.csv')

In [4]:
df["Date"] = pd.to_datetime(df["Date"])
test["Date"] = pd.to_datetime(test["Date"])

In [5]:
df["Latlong"] = zip(df["Latitude"], df["Longitude"])
test["Latlong"] = zip(test["Latitude"], test["Longitude"])

In [6]:
#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

station_1 = (41.995, -87.933)
station_2 = (41.786, -87.752)

print(vincenty(station_1, station_2).miles)

17.1810367501


In [7]:
def checkDistance(x):
    if vincenty(station_1,x).miles > vincenty(station_2,x).miles:
        return 2
    else:
        return 1

In [8]:
df["Station"] = df["Latlong"].apply(checkDistance)
test["Station"] = test["Latlong"].apply(checkDistance)

In [9]:
weather = pd.read_csv("../assets/weather.csv")

In [10]:
weather["Date"] = pd.to_datetime(weather["Date"])

In [11]:
df3 = df.merge(weather,on=["Date","Station"])
test3 = test.merge(weather,on=["Date","Station"])

In [12]:
def replaceNulls(x):
    if (x == "M") | (x==" ") | (x=="-"):
        return np.nan
    else:
        return x

In [13]:
df3 = df3.applymap(replaceNulls)
test3 = test3.applymap(replaceNulls)

In [14]:
df3['PrecipTotal'].fillna(value = 0, inplace = True)

In [15]:
df3['StnPressure'].fillna(value = 29.29, inplace=True)

In [16]:
df3.drop(["Depart","Depth","SnowFall","Water1","CodeSum","WetBulb"],axis=1, inplace=True)
test3.drop(["Depart","Depth","SnowFall","Water1","CodeSum","WetBulb"],axis=1, inplace=True)

In [17]:
weather.drop(["Depart","Depth","SnowFall","Water1","CodeSum","WetBulb"],axis=1, inplace=True)

In [18]:
def replaceTrace(x):
    if ("T" in str(x)):
        return .0001
    else:
        return x

In [19]:
df3['PrecipTotal'] = df3['PrecipTotal'].apply(replaceTrace)
test3['PrecipTotal']=test3['PrecipTotal'].apply(replaceTrace)
test = test3

In [20]:
df4 = df3.sort_values(by="Date").ffill().copy(deep=True)
test4 = test.sort_values(by="Date").ffill().copy(deep=True)

In [21]:
df2007 = df4[df4.Date.dt.year == 2007]
df2009 = df4[df4.Date.dt.year == 2009]
df2011 = df4[df4.Date.dt.year == 2011]
df2013 = df4[df4.Date.dt.year == 2013]

In [22]:
df2008 = test4[test4.Date.dt.year == 2008]
df2010 = test4[test4.Date.dt.year == 2010]
df2012 = test4[test4.Date.dt.year == 2012]
df2014 = test4[test4.Date.dt.year == 2014]

In [23]:
def score(model, name, myX,myy):
    s = cross_val_score(model, myX, myy, cv=cv, scoring = 'roc_auc')
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [24]:
def fitAndPrint(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print roc_auc_score(y_test,y_pred)

In [25]:
weather_list = ["AvgSpeed","Tavg","Heat","ResultDir","SeaLevel","Station","Sunrise","Sunset","Longitude","Latitude"]

In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(df4["Species"].values) + list(test4["Species"].values))

LabelEncoder()

In [27]:
df4["Species"] = le.transform(df4["Species"].values)
test4["Species"] = le.transform(test4["Species"].values)

In [28]:
le.fit(list(df4["Trap"].values) + list(test4["Trap"].values))
df4["Trap"] = le.transform(df4["Trap"].values)
test4["Trap"] = le.transform(test4["Trap"].values)

In [29]:
le.fit(list(df4["Street"].values) + list(test4["Street"].values))
df4["Street"] = le.transform(df4["Street"].values)
test4["Street"] = le.transform(test4["Street"].values)

In [30]:
species_dummies_2007 = pd.get_dummies(df2007.Species)
species_dummies_2009 = pd.get_dummies(df2009.Species)
species_dummies_2011 = pd.get_dummies(df2011.Species)
species_dummies_2013 = pd.get_dummies(df2013.Species)

species_dummies_2007.reset_index(drop=True,inplace=True)
species_dummies_2009.reset_index(drop=True,inplace=True)
species_dummies_2011.reset_index(drop=True,inplace=True)
species_dummies_2013.reset_index(drop=True,inplace=True)

In [31]:
df4.columns

Index([u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'NumMosquitos', u'WnvPresent', u'Latlong',
       u'Station', u'Tmax', u'Tmin', u'Tavg', u'DewPoint', u'Heat', u'Cool',
       u'Sunrise', u'Sunset', u'PrecipTotal', u'StnPressure', u'SeaLevel',
       u'ResultSpeed', u'ResultDir', u'AvgSpeed'],
      dtype='object')

In [32]:
x_list = (weather.columns - ["Date","Heat","Station"]).tolist()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


In [33]:
x_list

['AvgSpeed',
 'Cool',
 'DewPoint',
 'PrecipTotal',
 'ResultDir',
 'ResultSpeed',
 'SeaLevel',
 'StnPressure',
 'Sunrise',
 'Sunset',
 'Tavg',
 'Tmax',
 'Tmin']

In [34]:
df_combo = pd.concat([df4[["Species","Trap","Street","Block","Latitude","Longitude"]], df4[x_list]],axis=1)
df_combo_test = pd.concat([test4[["Species","Trap","Street","Block","Latitude","Longitude"]], test4[x_list]],axis=1)

In [35]:
df_combo_scaled = StandardScaler().fit_transform(df_combo)
df_combo_scaled_test = StandardScaler().fit_transform(df_combo_test)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_combo_scaled,df4["WnvPresent"],test_size=.70)

In [37]:
rf = RandomForestClassifier()

In [38]:
fitAndPrint(rf)

0.550080790303


In [39]:
for a,b in zip(rf.feature_importances_,df_combo.columns):
    print a,b

0.114659734662 Species
0.115680688915 Trap
0.0882209400994 Street
0.0806096651641 Block
0.0956662050154 Latitude
0.1188636899 Longitude
0.022883722881 AvgSpeed
0.0206220571625 Cool
0.0235916238143 DewPoint
0.0314106325815 PrecipTotal
0.0282160589378 ResultDir
0.0306141659511 ResultSpeed
0.0248607764937 SeaLevel
0.0216488240294 StnPressure
0.0541155642896 Sunrise
0.0612742997508 Sunset
0.0225683042692 Tavg
0.0218980869724 Tmax
0.022594959111 Tmin


In [40]:
predictions = rf.predict_proba(df_combo_scaled_test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('df_combo_rf.csv', index=False)

In [41]:
clf = svm.SVC(probability=True,kernel="linear")

In [42]:
fitAndPrint(clf)

0.5


In [44]:
predictions = clf.predict_proba(df_combo_scaled_test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('df_combo_clf.csv', index=False)

In [ ]:
gamma_range = 10.**np.arange(-3, 2)
C_range = 10.**np.arange(-1, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(clf, param_grid, cv=10, scoring='accuracy')

In [ ]:
fitAndPrint(grid)

In [ ]:
predictions = grid.predict_proba(df_combo_scaled_test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('df_combo_clf_grid.csv', index=False)

In [ ]:
X_weather_scaled = StandardScaler().fit_transform(df4[x_list])
X_weather_scaled_test = StandardScaler().fit_transform(test4[x_list])

In [ ]:
print X_weather_scaled.shape
print X_weather_scaled_test.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_weather_scaled,df4["WnvPresent"],test_size=.60)

In [ ]:
fitAndPrint(clf)

In [ ]:
predictions = clf.predict_proba(X_weather_scaled_test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('weather_clf_60.csv', index=False)

In [ ]:
X_species_weather = pd.concat([species_dummies, df4[x_list]],axis=1)

In [ ]:
X_weather_woheat_nummos = StandardScaler().fit_transform(df4[x2_list])

In [ ]:
X_species_weather_scaled = StandardScaler().fit_transform(X_species_weather)

In [ ]:
X_weather_scaled = StandardScaler().fit_transform(df4[x_list])

In [ ]:
X_weather_scaled_2007 = StandardScaler().fit_transform(df2007[x_list])

In [ ]:
X_weather_dropcorr = StandardScaler().fit_transform(df4[weather_list])

In [ ]:
x1_list = ['AvgSpeed',
 'Cool',
 'DewPoint',
 'ResultDir',
 'ResultSpeed',
 'SeaLevel',
 'Station',
 'Sunrise',
 'Sunset',
 'Tavg',
 'Tmax',
 'Tmin']

In [ ]:
X_weather_woheat_scaled = StandardScaler().fit_transform(df4[x1_list])

In [ ]:
pca = PCA(n_components=3)
X_species_weather_scaled_pca = pca.fit_transform(X_species_weather_scaled)
X_weather_scaled_pca = pca.fit_transform(X_weather_scaled)
X_weather_woheat_scaled_pca = pca.fit_transform(X_weather_woheat_scaled)

In [ ]:
trapm = df4.groupby(["Trap"])["NumMosquitos"].median()

In [ ]:
trapm = pd.DataFrame({"Trap":trapm.index,"NumMosquitos":trapm.values})

In [ ]:
trapm.head()

In [ ]:
test4 = pd.merge(test4,trapm,on="Trap", how="left")

In [ ]:
test4["NumMosquitos"].fillna(df4["NumMosquitos"].median(),inplace=True)

In [ ]:
y = df4["WnvPresent"]

In [ ]:
def score(model, name, myX,myy):
    s = cross_val_score(model, myX, myy, cv=cv)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [ ]:
def score_year(model, name, myX,myy):
    s = cross_val_score(model, myX, myy, cv=cvyear)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [ ]:
dt = DecisionTreeClassifier()
bdt = BaggingClassifier(DecisionTreeClassifier())
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
lr = LogisticRegression()

In [ ]:
def fitAndPrint(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print roc_auc_score(y_test,y_pred)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_species_weather_scaled,y)
fitAndPrint(lr)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_weather_dropcorr,y)
fitAndPrint(rf)

In [ ]:
rf.feature_importances_

In [ ]:
X_weather_test_scaled = StandardScaler().fit_transform(test4[weather_list])
predictions = rf.predict_proba(X_weather_test_scaled)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('weather_pred_rf_nummos.csv', index=False)